In [2]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/samplepesu/Training Dataset_csv/V_HELICOPTER_038_LABELS.csv
/kaggle/input/samplepesu/Training Dataset_csv/V_HELICOPTER_017_LABELS.csv
/kaggle/input/samplepesu/Training Dataset_csv/V_DRONE_025_LABELS.csv
/kaggle/input/samplepesu/Training Dataset_csv/V_DRONE_073_LABELS.csv
/kaggle/input/samplepesu/Training Dataset_csv/V_BIRD_007_LABELS.csv
/kaggle/input/samplepesu/Training Dataset_csv/V_HELICOPTER_012_LABELS.csv
/kaggle/input/samplepesu/Training Dataset_csv/V_AIRPLANE_041_LABELS.csv
/kaggle/input/samplepesu/Training Dataset_csv/V_HELICOPTER_033_LABELS.csv
/kaggle/input/samplepesu/Training Dataset_csv/V_BIRD_001_LABELS.csv
/kaggle/input/samplepesu/Training Dataset_csv/V_DRONE_051_LABELS.csv
/kaggle/input/samplepesu/Training Dataset_csv/V_DRONE_069_LABELS.csv
/kaggle/input/samplepesu/Training Dataset_csv/V_DRONE_043_LABELS.csv
/kaggle/input/samplepesu/Training Dataset_csv/V_DRONE_074_LABELS.csv
/kaggle/input/samplepesu/Training Dataset_csv/V_DRONE_008_LABELS.csv
/kaggle/input

In [70]:
import pandas as pd
import numpy as np
import cv2
import matplotlib.pyplot as plt
import os
from sklearn.model_selection import train_test_split
import csv

In [2]:

import pandas as pd
import numpy as np
import cv2
import matplotlib.pyplot as plt
import os
import numpy as np
csv_metadata_paths = os.listdir("/kaggle/input/samplepesu/Training Dataset_csv")
video_data_paths = os.listdir("/kaggle/input/samplepesu/Training Dataset_Videos")
video_data_folder = "/kaggle/input/samplepesu/Training Dataset_Videos"
csv_data_folder = "/kaggle/input/samplepesu/Training Dataset_csv"

os.mkdir("/kaggle/working/training_images_from_videos")
images_drop_folder = "/kaggle/working/training_images_from_videos"

i=0

length = len(video_data_paths)
not_done=[]
while i<length:
    try:
        #1 . get video

        video = cv2.VideoCapture(f"{video_data_folder}/{video_data_paths[i]}")

        #create folder for images to be dropped into 

        drop_images_in_subfolder = f"/kaggle/working/training_images_from_videos/{video_data_paths[i][:-4]}"
        os.mkdir(drop_images_in_subfolder)
        # 2. get fps
        fps = video.get(cv2.CAP_PROP_FPS)

        # 3. get images times
        df = pd.read_csv(f"{csv_data_folder}/{str(csv_metadata_paths[i])}")
        times=list(df.iloc[:,0])
        length = len(times)
        j=0
        while j<length:
            element = times[j]
            times[j] = element[1:-5]
            times[j] = float(times[j])
            j = j+1

        # frame ids of the required times are:
        frameids = []

        for element in times:

            frame_id = int(fps*element)
            frameids.append(frame_id)

        frameids = np.unique(frameids)
        # extracting the frame from the video

        for frame in frameids:
            video.set(cv2.CAP_PROP_POS_FRAMES, frame)
            ret, required_frame = video.read()
            cv2.imwrite(f'{drop_images_in_subfolder}/{frame}.png', required_frame)

        i=i+1
    except:
        not_done.append(i)
        i=i+1
        




In [105]:
import torch
import torch.nn as nn
import pandas as pd
import os
import PIL
import skimage
from skimage import io
import numpy as np
from PIL import Image
import matplotlib.pyplot as plt
import matplotlib.patches as patches
import torchvision.transforms as transforms
import torch.optim as optim
import torchvision.transforms.functional as FT
from torch.utils.data import DataLoader
from tqdm import tqdm
seed = 123
import cv2
import xml.etree.ElementTree as ET
torch.manual_seed(seed)
from collections import Counter

In [111]:
architecture_config = [
    #Tuple: (kernel_size, number of filters, strides, padding)
    (7, 64, 2, 3),
    #"M" = Max Pool Layer
    "M",
    (3, 192, 1, 1),
    "M",
    (1, 128, 1, 0),
    (3, 256, 1, 1),
    (1, 256, 1, 0),
    (3, 512, 1, 1),
    "M",
    #List: [(tuple), (tuple), how many times to repeat]
    [(1, 256, 1, 0), (3, 512, 1, 1), 4],
    (1, 512, 1, 0),
    (3, 1024, 1, 1),
    "M",
    [(1, 512, 1, 0), (3, 1024, 1, 1), 2],
    (3, 1024, 1, 1),
    (3, 1024, 2, 1),
    (3, 1024, 1, 1),
    (3, 1024, 1, 1),
    #Doesnt include fc layers
]

In [112]:
class CNNBlock(nn.Module):
    def __init__(self, in_channels, out_channels, **kwargs):
        super(CNNBlock, self).__init__()
        self.conv = nn.Conv2d(in_channels, out_channels, bias=False, **kwargs)
        self.batchnorm = nn.BatchNorm2d(out_channels)
        self.leakyrelu = nn.LeakyReLU(0.1)
        
    def forward(self, x):
        return self.leakyrelu(self.batchnorm(self.conv(x)))
    
class YoloV1(nn.Module):
    def __init__(self, in_channels=3, **kwargs):
        super(YoloV1, self).__init__()
        self.architecture = architecture_config
        self.in_channels = in_channels
        self.darknet = self._create_conv_layers(self.architecture)
        self.fcs = self._create_fcs(**kwargs)
        
    def forward(self, x):
        x = self.darknet(x)
        return self.fcs(torch.flatten(x, start_dim=1))
    
    def _create_conv_layers(self, architecture):
        layers = []
        in_channels = self.in_channels
        
        for x in architecture:
            if type(x) == tuple:
                layers += [CNNBlock(in_channels, x[1], kernel_size=x[0], stride=x[2], padding=x[3])]
                in_channels = x[1]
            elif type(x) == str:
                layers += [nn.MaxPool2d(kernel_size=2, stride=2)]
            elif type(x) == list:
                conv1 = x[0] #Tuple
                conv2 = x[1] #Tuple
                repeats = x[2] #Int
                
                for _ in range(repeats):
                    layers += [CNNBlock(in_channels, conv1[1], kernel_size=conv1[0], stride=conv1[2], padding=conv1[3])]
                    layers += [CNNBlock(conv1[1], conv2[1], kernel_size=conv2[0], stride=conv2[2], padding=conv2[3])]
                    in_channels = conv2[1]
                    
        return nn.Sequential(*layers)
    def _create_fcs(self, split_size, num_boxes, num_classes):
        S, B, C = split_size, num_boxes, num_classes
        return nn.Sequential(nn.Flatten(), nn.Linear(1024 * S * S, 496), nn.Dropout(0.0), nn.LeakyReLU(0.1), nn.Linear(496, S * S * (C + B * 5)))

In [113]:
def intersection_over_union(boxes_preds, boxes_labels, box_format='midpoint'):
    if box_format == 'midpoint':
        box1_x1 = boxes_preds[..., 0:1] - boxes_preds[..., 2:3] / 2
        box1_y1 = boxes_preds[..., 1:2] - boxes_preds[..., 3:4] / 2
        box1_x2 = boxes_preds[..., 0:1] + boxes_preds[..., 2:3] / 2
        box1_y2 = boxes_preds[..., 1:2] + boxes_preds[..., 3:4] / 2
        box2_x1 = boxes_labels[..., 0:1] - boxes_labels[..., 2:3] / 2
        box2_y1 = boxes_labels[..., 1:2] - boxes_labels[..., 3:4] / 2
        box2_x2 = boxes_labels[..., 0:1] + boxes_labels[..., 2:3] / 2
        box2_y2 = boxes_labels[..., 1:2] + boxes_labels[..., 3:4] / 2
        
    if box_format == 'corners':
        box1_x1 = boxes_preds[..., 0:1]
        box1_y1 = boxes_preds[..., 1:2]
        box1_x2 = boxes_preds[..., 2:3]
        box1_y2 = boxes_preds[..., 3:4] # Output tensor should be (N, 1). If we only use 3, we go to (N)
        box2_x1 = boxes_labels[..., 0:1]
        box2_y1 = boxes_labels[..., 1:2]
        box2_x2 = boxes_labels[..., 2:3]
        box2_y2 = boxes_labels[..., 3:4]
        
    x1 = torch.max(box1_x1, box2_x1)
    y1 = torch.max(box1_y1, box2_y1)
    x2 = torch.min(box1_x2, box2_x2)
    y2 = torch.min(box1_y2, box2_y2)
    
    intersection = (x2 - x1).clamp(0) * (y2 - y1).clamp(0)
    
    box1_area = abs((box1_x2 - box1_x1) * (box1_y2 - box1_y1))
    box2_area = abs((box2_x2 - box2_x1) * (box2_y2 - box2_y1))
    
    return intersection / (box1_area + box2_area - intersection + 1e-6)

In [114]:
def non_max_suppression(bboxes, iou_threshold, threshold, box_format="corners"):
    assert type(bboxes) == list

    bboxes = [box for box in bboxes if box[1] > threshold]
    bboxes = sorted(bboxes, key=lambda x: x[1], reverse=True)
    bboxes_after_nms = []

    while bboxes:
        chosen_box = bboxes.pop(0)

        bboxes = [
            box
            for box in bboxes
            if box[0] != chosen_box[0]
            or intersection_over_union(
                torch.tensor(chosen_box[2:]),
                torch.tensor(box[2:]),
                box_format=box_format,
            )
            < iou_threshold
        ]

        bboxes_after_nms.append(chosen_box)

    return bboxes_after_nms

In [115]:
def mean_average_precision(pred_boxes, true_boxes, iou_threshold=0.5, box_format="midpoint", num_classes=20):
    average_precisions = []

    epsilon = 1e-6

    for c in range(num_classes):
        detections = []
        ground_truths = []

        for detection in pred_boxes:
            if detection[1] == c:
                detections.append(detection)

        for true_box in true_boxes:
            if true_box[1] == c:
                ground_truths.append(true_box)

        amount_bboxes = Counter([gt[0] for gt in ground_truths])

        for key, val in amount_bboxes.items():
            amount_bboxes[key] = torch.zeros(val)

        detections.sort(key=lambda x: x[2], reverse=True)
        TP = torch.zeros((len(detections)))
        FP = torch.zeros((len(detections)))
        total_true_bboxes = len(ground_truths)
        
        if total_true_bboxes == 0:
            continue

        for detection_idx, detection in enumerate(detections):
            ground_truth_img = [bbox for bbox in ground_truths if bbox[0] == detection[0]]

            num_gts = len(ground_truth_img)
            best_iou = 0

            for idx, gt in enumerate(ground_truth_img):
                iou = intersection_over_union(
                    torch.tensor(detection[3:]),
                    torch.tensor(gt[3:]),
                    box_format=box_format,
                )

                if iou > best_iou:
                    best_iou = iou
                    best_gt_idx = idx

            if best_iou > iou_threshold:
                if amount_bboxes[detection[0]][best_gt_idx] == 0:
                    TP[detection_idx] = 1
                    amount_bboxes[detection[0]][best_gt_idx] = 1
                else:
                    FP[detection_idx] = 1

            else:
                FP[detection_idx] = 1

        TP_cumsum = torch.cumsum(TP, dim=0)
        FP_cumsum = torch.cumsum(FP, dim=0)
        recalls = TP_cumsum / (total_true_bboxes + epsilon)
        precisions = torch.divide(TP_cumsum, (TP_cumsum + FP_cumsum + epsilon))
        precisions = torch.cat((torch.tensor([1]), precisions))
        recalls = torch.cat((torch.tensor([0]), recalls))
        average_precisions.append(torch.trapz(precisions, recalls))

    return sum(average_precisions) / len(average_precisions)

In [116]:
def get_bboxes(
    loader,
    model,
    iou_threshold,
    threshold,
    pred_format="cells",
    box_format="midpoint",
    device="cuda",
):
    all_pred_boxes = []
    all_true_boxes = []
    # make sure model is in eval before get bboxes
    model.eval()
    train_idx = 0

    for batch_idx, (x, labels) in enumerate(loader):
        x = x.to(device)
        labels = labels.to(device)

        with torch.no_grad():
            predictions = model(x)

        batch_size = x.shape[0]
        true_bboxes = cellboxes_to_boxes(labels)
        bboxes = cellboxes_to_boxes(predictions)

        for idx in range(batch_size):
            nms_boxes = non_max_suppression(
                bboxes[idx],
                iou_threshold=iou_threshold,
                threshold=threshold,
                box_format=box_format,
            )
            #if batch_idx == 0 and idx == 0:
            #    plot_image(x[idx].permute(1,2,0).to("cpu"), nms_boxes)
            #    print(nms_boxes)

            for nms_box in nms_boxes:
                all_pred_boxes.append([train_idx] + nms_box)

            for box in true_bboxes[idx]:
                # many will get converted to 0 pred
                if box[1] > threshold:
                    all_true_boxes.append([train_idx] + box)

            train_idx += 1

    model.train()
    return all_pred_boxes, all_true_boxes

def convert_cellboxes(predictions, S=7, C=3):
    predictions = predictions.to("cpu")
    batch_size = predictions.shape[0]
    predictions = predictions.reshape(batch_size, 7, 7, C + 10)
    bboxes1 = predictions[..., C + 1:C + 5]
    bboxes2 = predictions[..., C + 6:C + 10]
    scores = torch.cat(
        (predictions[..., C].unsqueeze(0), predictions[..., C + 5].unsqueeze(0)), dim=0
    )
    best_box = scores.argmax(0).unsqueeze(-1)
    best_boxes = bboxes1 * (1 - best_box) + best_box * bboxes2
    cell_indices = torch.arange(7).repeat(batch_size, 7, 1).unsqueeze(-1)
    x = 1 / S * (best_boxes[..., :1] + cell_indices)
    y = 1 / S * (best_boxes[..., 1:2] + cell_indices.permute(0, 2, 1, 3))
    w_y = 1 / S * best_boxes[..., 2:4]
    converted_bboxes = torch.cat((x, y, w_y), dim=-1)
    predicted_class = predictions[..., :C].argmax(-1).unsqueeze(-1)
    best_confidence = torch.max(predictions[..., C], predictions[..., C + 5]).unsqueeze(
        -1
    )
    converted_preds = torch.cat(
        (predicted_class, best_confidence, converted_bboxes), dim=-1
    )
    return converted_preds

def cellboxes_to_boxes(out, S=7):
    converted_pred = convert_cellboxes(out).reshape(out.shape[0], S * S, -1)
    converted_pred[..., 0] = converted_pred[..., 0].long()
    all_bboxes = []

    for ex_idx in range(out.shape[0]):
        bboxes = []

        for bbox_idx in range(S * S):
            bboxes.append([x.item() for x in converted_pred[ex_idx, bbox_idx, :]])
        all_bboxes.append(bboxes)

    return all_bboxes

def save_checkpoint(state, filename="my_checkpoint.pth"):
    print("=> Saving checkpoint")
    torch.save(state, filename)
    
def load_checkpoint(checkpoint, model, optimizer):
    print("=> Loading checkpoint")
    model.load_state_dict(checkpoint["state_dict"])
    optimizer.load_state_dict(checkpoint["optimizer"])

In [61]:
test_dir = '../input/fruit-images-for-object-detection/test_zip/test'

folder_of_folders = "/kaggle/working/training_images_from_videos"

global_df = pd.DataFrame()
for files_dir in os.listdir(folder_of_folders):
    annotation = "airplane"
    if(files_dir.startswith("V_BIRD")):
        annotation = "bird"
    elif(files_dir.startswith("V_HELICOPTER")):
        annotation = "helicopter"
    elif(files_dir.startswith("V_DRONE")):
        annotation = "drone"
    #print(annotation)
    
    files_path = folder_of_folders+'/'+files_dir
    images = [files_path+'/'+image for image in sorted(os.listdir(files_path))]
    record_no = [image[:-4] for image in sorted(os.listdir(files_path))]
    annots = []
    
    files_dirs = []
    for image in images:
        files_dirs.append(files_dir)
        annots.append(annotation)
    
    images = pd.Series(images, name='images')
    annots = pd.Series(annots, name='annots')
    files_dirs = pd.Series(files_dirs, name='files_dir')
    record_no = pd.Series(record_no, name='record_no')
    df = pd.concat([files_dirs, images, annots, record_no], axis=1)
    df = pd.DataFrame(df)
    global_df = global_df.append(df)
    
global_df.describe()

/opt/conda/lib/python3.7/site-packages/ipykernel_launcher.py:26: DeprecationWarning: The default dtype for empty Series will be 'object' instead of 'float64' in a future version. Specify a dtype explicitly to silence this warning.
/opt/conda/lib/python3.7/site-packages/ipykernel_launcher.py:27: DeprecationWarning: The default dtype for empty Series will be 'object' instead of 'float64' in a future version. Specify a dtype explicitly to silence this warning.
/opt/conda/lib/python3.7/site-packages/ipykernel_launcher.py:28: DeprecationWarning: The default dtype for empty Series will be 'object' instead of 'float64' in a future version. Specify a dtype explicitly to silence this warning.
/opt/conda/lib/python3.7/site-packages/ipykernel_launcher.py:29: DeprecationWarning: The default dtype for empty Series will be 'object' instead of 'float64' in a future version. Specify a dtype explicitly to silence this warning.


,files_dir,images,annots,record_no
count,56189,56189,56189,56189
unique,274,56189,4,219
top,V_DRONE_092,/kaggle/working/training_images_from_videos/V_...,drone,246
freq,219,1,22276,274


In [62]:
global_df.reset_index(inplace=True, drop=True)
train, test = train_test_split(global_df, test_size=0.3, random_state=42, shuffle=True)
train.reset_index(inplace=True, drop=True)
test.reset_index(inplace=True, drop=True)
train.head()

,files_dir,images,annots,record_no
0,V_AIRPLANE_049,/kaggle/working/training_images_from_videos/V_...,airplane,5
1,V_BIRD_042,/kaggle/working/training_images_from_videos/V_...,bird,246
2,V_HELICOPTER_057,/kaggle/working/training_images_from_videos/V_...,helicopter,194
3,V_DRONE_081,/kaggle/working/training_images_from_videos/V_...,drone,51
4,V_HELICOPTER_036,/kaggle/working/training_images_from_videos/V_...,helicopter,128


In [63]:
train.iloc[0]

files_dir                                       V_AIRPLANE_049
images       /kaggle/working/training_images_from_videos/V_...
annots                                                airplane
record_no                                                    5
Name: 0, dtype: object

In [64]:
print(train.iloc[0]["files_dir"])
print(train.iloc[0]["images"])
print(train.iloc[0]["annots"])
print(train.iloc[0]["record_no"])

V_AIRPLANE_049
/kaggle/working/training_images_from_videos/V_AIRPLANE_049/5.png
airplane
5


In [65]:
test.head()

,files_dir,images,annots,record_no
0,V_DRONE_062,/kaggle/working/training_images_from_videos/V_...,drone,122
1,V_DRONE_057,/kaggle/working/training_images_from_videos/V_...,drone,26
2,V_BIRD_001,/kaggle/working/training_images_from_videos/V_...,bird,59
3,V_HELICOPTER_012,/kaggle/working/training_images_from_videos/V_...,helicopter,270
4,V_HELICOPTER_038,/kaggle/working/training_images_from_videos/V_...,helicopter,29


In [66]:
train.describe()

,files_dir,images,annots,record_no
count,39332,39332,39332,39332
unique,274,39332,4,219
top,V_DRONE_089,/kaggle/working/training_images_from_videos/V_...,drone,174
freq,163,1,15716,211


In [67]:
test.describe()

,files_dir,images,annots,record_no
count,16857,16857,16857,16857
unique,274,16857,4,218
top,V_AIRPLANE_017,/kaggle/working/training_images_from_videos/V_...,drone,11
freq,81,1,6560,106


In [68]:
df = train
test_df = test

In [137]:
class AviationImagesDataset(torch.utils.data.Dataset):
    def __init__(self, df=df, S=7, B=2, C=3, transform=None):
        self.annotations = df
        self.transform = transform
        self.S = S
        self.B = B
        self.C = C

    def __len__(self):
        return len(self.annotations)

    def __getitem__(self, index):
        record = self.annotations.iloc[index]
        csv_file = "/kaggle/input/samplepesu/Training Dataset_csv/"+self.annotations.iloc[index]["files_dir"]+"_LABELS.csv"
        boxes = []
        class_dictionary = {'airplane':0, 'bird':1, 'drone':2, 'helicopter':3}
        
        with open(csv_file) as file_obj:
            reader_obj = csv.reader(file_obj)
            curr = 0
            for row in reader_obj:
                if(curr == record["record_no"]):
                    objs = row[1].split("],[")
                    for obj in objs:
                        obj = obj.strip("[]").split(",")
                        obj = [float(x.strip()) for x in obj]
                        obj.insert(0,class_dictionary[self.annotations.iloc[index]["annots"]])
                        boxes.append(obj)
                    break
                else:
                    curr+=1
        
        img_path = self.annotations.iloc[index]["images"]
        image = Image.open(img_path)
        boxes = torch.tensor(boxes)
        
        if self.transform:
            image, boxes = self.transform(image, boxes)
        
        label_matrix = torch.zeros((self.S, self.S, self.C + 5 * self.B))
        for box in boxes:
            class_label, x, y, width, height = box.tolist()
            class_label = int(class_label)
            i, j = int(self.S * y), int(self.S * x)
            x_cell, y_cell = self.S * x - j, self.S * y - i
            
            width_cell, height_cell = (
                width * self.S,
                height * self.S,
            )

            if label_matrix[i, j, self.C] == 0:
                label_matrix[i, j, self.C] = 1
                box_coordinates = torch.tensor(
                    [x_cell, y_cell, width_cell, height_cell]
                )

                label_matrix[i, j, 4:8] = box_coordinates
                label_matrix[i, j, class_label] = 1
        return image, label_matrix

In [104]:
csv_file = "/kaggle/input/samplepesu/Training Dataset_csv/V_AIRPLANE_001_LABELS.csv"
boxes = []
with open(csv_file) as file_obj:
    reader_obj = csv.reader(file_obj)
    print(reader_obj)
    curr = 0
    index = 2
    boxes = []
    for row in reader_obj:
        if(curr == index):
            print(row)
            print(row[1])
            objs = row[1].split("],[")
            for obj in objs:
                obj = obj.strip("[]").split(",")
                obj = [float(x) for x in obj]
                obj.insert(0,0)
                boxes.append(obj)
            break
        else:
            curr+=1
    print(boxes)
    
    
    

["'0.066667 sec'", '[311.489379882813,204.399459838867,36.0547180175781,24.1059265136719]', '[]', '[]', '[]']
[311.489379882813,204.399459838867,36.0547180175781,24.1059265136719]
[[0, 311.489379882813, 204.399459838867, 36.0547180175781, 24.1059265136719]]


In [107]:
class YoloLoss(nn.Module):

    def __init__(self, S=7, B=2, C=3):
        super(YoloLoss, self).__init__()
        self.mse = nn.MSELoss(reduction="sum")

        self.S = S
        self.B = B
        self.C = C

        self.lambda_noobj = 0.5
        self.lambda_coord = 5

    def forward(self, predictions, target):
        predictions = predictions.reshape(-1, self.S, self.S, self.C + self.B * 5)

        iou_b1 = intersection_over_union(predictions[..., self.C + 1:self.C + 5], target[..., self.C + 1:self.C + 5])
        iou_b2 = intersection_over_union(predictions[..., self.C + 6:self.C + 10], target[..., self.C + 1:self.C + 5])
        ious = torch.cat([iou_b1.unsqueeze(0), iou_b2.unsqueeze(0)], dim=0)

        iou_maxes, bestbox = torch.max(ious, dim=0)
        exists_box = target[..., self.C].unsqueeze(3)  

        box_predictions = exists_box * (
            (
                bestbox * predictions[..., self.C + 6:self.C + 10]
                + (1 - bestbox) * predictions[..., self.C + 1:self.C + 5]
            )
        )

        box_targets = exists_box * target[..., self.C + 1:self.C + 5]

        box_predictions[..., 2:4] = torch.sign(box_predictions[..., 2:4]) * torch.sqrt(
            torch.abs(box_predictions[..., 2:4] + 1e-6)
        )
        box_targets[..., 2:4] = torch.sqrt(box_targets[..., 2:4])

        box_loss = self.mse(
            torch.flatten(box_predictions, end_dim=-2),
            torch.flatten(box_targets, end_dim=-2),
        )

        pred_box = (
            bestbox * predictions[..., self.C + 5:self.C + 6] + (1 - bestbox) * predictions[..., self.C:self.C + 1]
        )

        object_loss = self.mse(
            torch.flatten(exists_box * pred_box),
            torch.flatten(exists_box * target[..., self.C:self.C + 1]),
        )

        no_object_loss = self.mse(
            torch.flatten((1 - exists_box) * predictions[..., self.C:self.C + 1], start_dim=1),
            torch.flatten((1 - exists_box) * target[..., self.C:self.C + 1], start_dim=1),
        )

        no_object_loss += self.mse(
            torch.flatten((1 - exists_box) * predictions[..., self.C + 5:self.C + 6], start_dim=1),
            torch.flatten((1 - exists_box) * target[..., self.C:self.C + 1], start_dim=1)
        )

        class_loss = self.mse(
            torch.flatten(exists_box * predictions[..., :self.C], end_dim=-2,),
            torch.flatten(exists_box * target[..., :self.C], end_dim=-2,),
        )

        loss = (
            self.lambda_coord * box_loss  # first two rows in paper
            + object_loss  # third row in paper
            + self.lambda_noobj * no_object_loss  # forth row
            + class_loss  # fifth row
        )

        return loss

In [108]:
LEARNING_RATE = 2e-5
DEVICE = "cuda" if torch.cuda.is_available() else "cpu"
BATCH_SIZE = 16 # 64 in original paper but resource exhausted error otherwise.
WEIGHT_DECAY = 0
EPOCHS = 20
NUM_WORKERS = 2
PIN_MEMORY = True
LOAD_MODEL = False
LOAD_MODEL_FILE = "model.pth"

In [109]:
def train_fn(train_loader, model, optimizer, loss_fn):
    loop = tqdm(train_loader, leave=True)
    mean_loss = []
    
    for batch_idx, (x, y) in enumerate(loop):
        x, y = x.to(DEVICE), y.to(DEVICE)
        out = model(x)
        loss = loss_fn(out, y)
        mean_loss.append(loss.item())
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        
        loop.set_postfix(loss = loss.item())
        
    print(f"Mean loss was {sum(mean_loss) / len(mean_loss)}")

In [110]:
class Compose(object):
    def __init__(self, transforms):
        self.transforms = transforms

    def __call__(self, img, bboxes):
        for t in self.transforms:
            img, bboxes = t(img), bboxes

        return img, bboxes


transform = Compose([transforms.Resize((448, 448)), transforms.ToTensor()])

In [ ]:
def main():
    model = YoloV1(split_size=7, num_boxes=2, num_classes=3).to(DEVICE)
    optimizer = optim.Adam(
        model.parameters(), lr=LEARNING_RATE, weight_decay=WEIGHT_DECAY
    )
    scheduler = optim.lr_scheduler.ReduceLROnPlateau(optimizer=optimizer, factor=0.1, patience=3, mode='max', verbose=True)
    loss_fn = YoloLoss()

    if LOAD_MODEL:
        load_checkpoint(torch.load(LOAD_MODEL_FILE), model, optimizer)

    train_dataset = AviationImagesDataset(
        transform=transform,
    )

    test_dataset = AviationImagesDataset(
        transform=transform, 
    )
    
    train_loader = DataLoader(
        dataset=train_dataset,
        batch_size=BATCH_SIZE,
        shuffle=True,
        drop_last=False,
    )

    test_loader = DataLoader(
        dataset=test_dataset,
        batch_size=BATCH_SIZE,
        shuffle=True,
        drop_last=False,
    )
    
    for epoch in range(EPOCHS):
        train_fn(train_loader, model, optimizer, loss_fn)
        
        pred_boxes, target_boxes = get_bboxes(
            train_loader, model, iou_threshold=0.5, threshold=0.4
        )

        mean_avg_prec = mean_average_precision(
            pred_boxes, target_boxes, iou_threshold=0.5, box_format="midpoint"
        )
        print(f"Train mAP: {mean_avg_prec}")
        
        scheduler.step(mean_avg_prec)
    
    checkpoint = {
            "state_dict": model.state_dict(),
            "optimizer": optimizer.state_dict(),
    }
    save_checkpoint(checkpoint, filename=LOAD_MODEL_FILE)
    
if __name__ == "__main__":
    main()

100%|██████████| 2459/2459 [27:14<00:00,  1.50it/s, loss=0.00106]


Mean loss was 0.07925484805349328
